## 코딩테스트

### MNIST 손글씨 숫자 분류

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 데이터 로딩 및 전처리

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'CPU/GPU: {device}')

transform = transforms.ToTensor()

train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

CPU/GPU: cuda


### CNN 모델 선언

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # 문제. 아래의 주석을 풀고 알맞은 Flatten 내용을 추가하시오.
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
model = CNN()

### 손실함수와 옵티마이저

In [9]:
criterion = nn.CrossEntropyLoss()
# 문제: 아래의 주석을 해제, Adam 옵티마이저를 설정하고 학습률은 0.001로 지정하세요.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### 학습

In [10]:
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        # 문제: 아래의 주석 해제, 손실(loss)를 계산하세요.
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")

[Epoch 1] Loss: 0.2261
[Epoch 2] Loss: 0.0625
[Epoch 3] Loss: 0.0425
[Epoch 4] Loss: 0.0325
[Epoch 5] Loss: 0.0261


### 평가 및 저장

In [12]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        # 문제: 주석해제 후, 정확히 예측한 개수를 누적하세요.
        correct += (predicted == labels).sum().item()
print(f'테스트 정확도: {100 * correct / total:.2f}%')

# 문제: 모델 저장 (선택) - 학습된 모델을 'mnist_cnn.pth' 파일로 저장하는 코드를 작성하시오
torch.save(model, 'mnist_cnn.pth')

테스트 정확도: 98.92%


#### 위의 내용을 CUDA에서 동작하도록 하나의 셀에 작성 완성하시오

In [14]:
# 문제: 전체 소스 작성

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'CPU/GPU: {device}')

# 데이터 변환
transform = transforms.ToTensor()

# MNIST 데이터셋 로딩
train_set = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform)

test_set = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# CNN 모델 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.fc1 = nn.Linear(32 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 생성 및 GPU로 이동
model = CNN().to(device)

# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")

# 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'테스트 정확도: {100 * correct / total:.2f}%')

# 모델 저장
torch.save(model.state_dict(), 'mnist_cnn.pth')


CPU/GPU: cuda
[Epoch 1] Loss: 0.2427
[Epoch 2] Loss: 0.0626
[Epoch 3] Loss: 0.0451
[Epoch 4] Loss: 0.0335
[Epoch 5] Loss: 0.0279
테스트 정확도: 98.39%
